imagefolder

In [72]:
import torch
import torchvision
import visdom

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [73]:
vis = visdom.Visdom()
vis.close(env='main')

Setting up a new session...


''

In [74]:
# train_data = torchvision.datasets.ImageFolder(
#   root='icons/train_data_original', 
#   transform=torchvision.transforms.Resize((64, 64))
#)
train_data = torchvision.datasets.ImageFolder(
    root='icons/train_data_resized', 
    transform=torchvision.transforms.ToTensor()
)
data_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size=16, shuffle=True)

In [75]:
test_batch_size = 16
name_list = ['Facebook', 'Google', 'Instagram', 'Youtube']
test_data = torchvision.datasets.ImageFolder(
    root='icons/test_data', 
    transform=torchvision.transforms.ToTensor()
)
test_data_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size=test_batch_size)

In [76]:
'''
for num, value in enumerate(train_data):
    data, label = value
    print(num, data, label)

    if label == 0:
        data.save('icons/train_data_resized/facebook/%d_%d.png'%(num, label))
    elif label == 1:
        data.save('icons/train_data_resized/google/%d_%d.png'%(num, label))
    elif label == 2:
        data.save('icons/train_data_resized/instagram/%d_%d.png'%(num, label))
    else:
        data.save('icons/train_data_resized/youtube/%d_%d.png'%(num, label))
'''

"\nfor num, value in enumerate(train_data):\n    data, label = value\n    print(num, data, label)\n\n    if label == 0:\n        data.save('icons/train_data_resized/facebook/%d_%d.png'%(num, label))\n    elif label == 1:\n        data.save('icons/train_data_resized/google/%d_%d.png'%(num, label))\n    elif label == 2:\n        data.save('icons/train_data_resized/instagram/%d_%d.png'%(num, label))\n    else:\n        data.save('icons/train_data_resized/youtube/%d_%d.png'%(num, label))\n"

In [77]:
class CNN(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 8, kernel_size=5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2)
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(8, 16, kernel_size=3),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2)
        )
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(16, 32, kernel_size=3),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2)
        )

        self.fc1 = torch.nn.Linear(32*6*6, 1024, bias=True)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(1024, 120, bias=True)
        self.fc3 = torch.nn.Linear(120, 4, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)

        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        
        return out

    

In [78]:
net = CNN().to(device)

In [79]:
optimizer = torch.optim.Adam(net.parameters(), lr = 1e-4)
loss_func = torch.nn.CrossEntropyLoss().to(device)

In [80]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='loss_tracker', legend=['loss'], showlegend=True), env='main')
test_loss_plt = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='test_loss_tracker', legend=['loss'], showlegend=True), env='main')
accuracy_plt = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='accuracy_tracker', legend=['accuracy'], showlegend=True), env='main')


def graph_appender(loss_plot, loss_value, num):
    ''' num, loss_value are Tensor '''
    vis.line(X=num, Y=loss_value, win=loss_plot, update="append")

In [81]:
def test(test_data_load, idx, print_image = False):
    for num, data in enumerate(test_data_load):
        img, label = data
        img = img.view(-1, 3, 64, 64).to(device)
        label = label.to(device)
        out = net(img)

        prob = torch.max(torch.nn.Softmax(dim=1)(out) * 100, dim=1)
        loss = loss_func(out, label)
        
        accuracy = (prob[1] == label).float().mean()
        if print_image:
            for i in range(test_batch_size):
                vis.image(img[i], opts={'height' : 250, 'width' : 250, 'title' : ('image ' + str(i)), 'caption' : ('prediction : ' + name_list[prob[1][i]] + '(' + '{:.2f}'.format(prob[0][i].item()) + '%), label : ' + name_list[label[i]])}, env='main')

        graph_appender(test_loss_plt, torch.Tensor([loss.item()]), torch.Tensor([idx]))
        graph_appender(accuracy_plt, torch.Tensor([accuracy.item() * 100]), torch.Tensor([idx]))

In [82]:
total_batch = len(data_loader)

epochs = 100
for epoch in range(epochs):
    avg_cost = 0.0
    for num, data in enumerate(data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        out = net(imgs)
        loss = loss_func(out, labels)
        loss.backward()
        optimizer.step()

        avg_cost += loss / total_batch
    
    print('[Epoch:{:2d}] cost = {}'.format(epoch + 1, avg_cost))
    graph_appender(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))

    test(test_data_loader, epoch)
print('Learning Finished!')

[Epoch: 1] cost = 1.3856420516967773
[Epoch: 2] cost = 1.3675309419631958
[Epoch: 3] cost = 1.354541540145874
[Epoch: 4] cost = 1.3395830392837524
[Epoch: 5] cost = 1.3208688497543335
[Epoch: 6] cost = 1.3002783060073853
[Epoch: 7] cost = 1.2744436264038086
[Epoch: 8] cost = 1.2437033653259277
[Epoch: 9] cost = 1.2054898738861084
[Epoch:10] cost = 1.1658623218536377
[Epoch:11] cost = 1.1152877807617188
[Epoch:12] cost = 1.0615448951721191
[Epoch:13] cost = 0.9935574531555176
[Epoch:14] cost = 0.9274929165840149
[Epoch:15] cost = 0.8565378189086914
[Epoch:16] cost = 0.7875544428825378
[Epoch:17] cost = 0.7176052927970886
[Epoch:18] cost = 0.6610107421875
[Epoch:19] cost = 0.6110310554504395
[Epoch:20] cost = 0.5609748959541321
[Epoch:21] cost = 0.5325257182121277
[Epoch:22] cost = 0.4942832589149475
[Epoch:23] cost = 0.4910900294780731
[Epoch:24] cost = 0.44812431931495667
[Epoch:25] cost = 0.42189347743988037
[Epoch:26] cost = 0.4019445776939392
[Epoch:27] cost = 0.389055073261261
[Epo

In [83]:
test(test_data_loader, epoch, True)
test(data_loader, 101, True)